In [1]:
from IPython.display import display_html
display_html("""<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">Hide Code</button>""", raw=True)

Hide Code

# IV. Objective 3: Recommendation System (Ni)
In this part, I will build recommendation system with trying three popular recommendation methods and one deep learning approach(Artificial Nerual Network). The target of this part is to recommend 3 more different restaurants to each customer in the Hoboken Restaurants Reviews dataset. The recommendation system will help business owner as well as yelp to create profit. The methods I will try showed as follows:
1. Co-occurrence Matrix 
2. Collaborative Filtering
3. Matrix Factorization
4. Deep learning (Neural Network)

In [1]:
#py3.6
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import Normalizer
import itertools
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [1]:
#py2.7
import graphlab 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split

## Part 1. Data Understanding and Cleaning
Before building the model, I will introduce the data I will use. The raw data in this part is the Hoboken Restaurants Reviews dataset, which includes 74,611 rows and 8 columns. First, I will clean the data and prepare required data for following recommendation system methods.

### Part 1 - 1 Basic Understanding of data
Definition of each variable:
1. user_id: Unique id for each customer
2. user_name: customer's name
3. user_raing: original rating for one restaurant per review
4. user_text: text of customer's review for one restaurant per review 
5. restaurant_name: unique name for each restaurant
6. restaurant_rating: the integrated rating of each restaurant
7. restaurant_price: degree of cheap or expensive of one restaurant
8. restaurant_type: the style and theme of one restaurant

**Table 1** show a sample dataset and  basic information of raw data. **Chart 1** shows the datatype of each column. There are some columns like user_rating, restaurant_rating and restaurant_price should be convert into numerical data. In addition, there are some repeated comma in restaurant_type column,  so I will do data cleaning first and try to use this variable in Neural Network part. In addition, it use '$$' symbol represent the degree of price of each restaurant and the symbol should be converted to integer, which could be recognized by the computer.

In [3]:
df=pd.read_csv('Hoboken_restaurants_reviews.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,user_id,user_name,user_rating,user_text,restaurant_name,restaurant_rating,restaurant_price,restaurant_type
0,0,0,dRuCO4NYO7zyAF8-CeJmZg,Jason L.,5.0 star rating,We booked Grand Vin as our brunch location to ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
1,1,1,f36YZ1cA291bNtMHXWtu1Q,Danyale W.,4.0 star rating,Sooooo for date night it was his turn to pick ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
2,2,2,-xYUKfWQTaB-7BeizsQA3w,Robin G.,5.0 star rating,Adorable little wine bar with outdoor seating ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
3,3,3,tt1vLgAP5UpRXAKJLT2KWg,Alec K.,4.0 star rating,One of the top restaurants in Hoboken. Well ma...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
4,4,4,-K79Xep4lElqIChsJYWuiQ,Robbie O.,5.0 star rating,Great space- service is on point - short rib ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"


In [4]:
del df['Unnamed: 0']
del df['Unnamed: 0.1']
df.head()

,user_id,user_name,user_rating,user_text,restaurant_name,restaurant_rating,restaurant_price,restaurant_type
0,dRuCO4NYO7zyAF8-CeJmZg,Jason L.,5.0 star rating,We booked Grand Vin as our brunch location to ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
1,f36YZ1cA291bNtMHXWtu1Q,Danyale W.,4.0 star rating,Sooooo for date night it was his turn to pick ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
2,-xYUKfWQTaB-7BeizsQA3w,Robin G.,5.0 star rating,Adorable little wine bar with outdoor seating ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
3,tt1vLgAP5UpRXAKJLT2KWg,Alec K.,4.0 star rating,One of the top restaurants in Hoboken. Well ma...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"
4,-K79Xep4lElqIChsJYWuiQ,Robbie O.,5.0 star rating,Great space- service is on point - short rib ...,Grand Vin,4.0 star rating,$$,"Wine, Bars,, Italian,, Cocktail, Bars"


<center>**Table 1: Sample of raw data**</center>

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74611 entries, 0 to 74610
Data columns (total 8 columns):
user_id              74611 non-null object
user_name            74611 non-null object
user_rating          74611 non-null object
user_text            74611 non-null object
restaurant_name      74611 non-null object
restaurant_rating    74611 non-null object
restaurant_price     74479 non-null object
restaurant_type      74611 non-null object
dtypes: object(8)
memory usage: 4.6+ MB


**Chart 1: Datatype**

We could find some basic information of the raw data from **Table 2**. It shows that there are 74,611 reviews in total, while only 44,949 unique users leave the reviews, which means multiple users attended to more than one restaurants and it is valuable to build the recommendation system. Also, it shows that there are only 24,781 unique user's name, so in order to reduce the ambigurity, I will use user_id instead of user name in the recommendation system.

In [6]:
df.describe()

,user_id,user_name,user_rating,user_text,restaurant_name,restaurant_rating,restaurant_price,restaurant_type
count,74611,74611,74611,74611,74611,74611,74479,74611
unique,44949,24781,5,69542,302,7,4,230
top,QxTNaHoxTL8f7jAK5nwQ1g,Nicholas G.,5.0 star rating,I've been coming to STK downtown since they ha...,Morimoto,4.0 star rating,$$,"Japanese,, Sushi, Bars"
freq,101,103,27702,3,2740,31568,38323,3620


<center>**Table 2: Basic Information**</center>

After checing the null value, we could find that there are null value in restaurant_price column. **Table 3** shows that there are some missing restaurant_price for some restaurants, since yelp have not updated the price for these restaurants. In following part, I will refill these missing value accordindly.

In [7]:
df.isnull().any()

user_id              False
user_name            False
user_rating          False
user_text            False
restaurant_name      False
restaurant_rating    False
restaurant_price      True
restaurant_type      False
dtype: bool

In [8]:
df[df.restaurant_price.isnull() == True]

,user_id,user_name,user_rating,user_text,restaurant_name,restaurant_rating,restaurant_price,restaurant_type
5273,i_WxJpoxfsddmBne6I8cKQ,Durva L.,4.0 star rating,"This is such a small place, but some great rea...",Funjabi,2.5 star rating,NaN,"Indian,, Chinese"
5274,zr0jkmEJLnaZxWZi7cp09Q,Wasbir R.,1.0 star rating,Giving 1 star because theres no 0 star. This i...,Funjabi,2.5 star rating,NaN,"Indian,, Chinese"
6782,cnNddUd4fn7h-Pb_Cma-9Q,mako y.,5.0 star rating,Wonderful!!! It was traditional with unique tw...,奥田,4.5 star rating,NaN,Japanese
6783,q1P19WvpTXFTmfU7oXgkag,Chester G.,2.0 star rating,When I heard Chef Toru Okuda was going to open...,奥田,4.5 star rating,NaN,Japanese
6784,hIrk8azL929_c4H4-3lEAg,Christie K.,5.0 star rating,Soooo good. A must try if you can appreciate g...,奥田,4.5 star rating,NaN,Japanese
6785,NSmWDNSyjqhm1vo7VUxL5w,Jay C.,5.0 star rating,It is with mixed emotion I write what follows....,奥田,4.5 star rating,NaN,Japanese
6786,xM30Cj7zdsTiRQ-zmAFNjw,Victor O.,5.0 star rating,It was memorable experience. It was hard to ma...,奥田,4.5 star rating,NaN,Japanese
6787,fbv2hE02tCljub9JcDSiDw,Cinesina C.,5.0 star rating,I travel a lot to Japan for work so when I rea...,奥田,4.5 star rating,NaN,Japanese
9348,7CvAUqZqqZBKvkgwI0kfzw,Steven P.,5.0 star rating,"Sad to hear they closed in The Heights, then t...",Mision Burrito,2.5 star rating,NaN,"Mexican,, Grocery,, Convenience, Stores"
9349,-jNxEJzXNWtbCy5VEyS3Lw,Mark A.,5.0 star rating,What a awesome place!! It's located on the cor...,Mision Burrito,2.5 star rating,NaN,"Mexican,, Grocery,, Convenience, Stores"


In [9]:
print('number of missing value:',len(df[df.restaurant_price.isnull() == True]))

number of missing value: 132


<center>**Table 3: Missing value**</center>

### Part1 - 2 Data Cleaning and Preprocessing
In this part, I will check each column one by one and prepare the dataset for recommendation system part.
#### - user_id
**Table 4** list the top 10 users who left most reviews to restaurants in Hoboken. Multiple customers left reviews for different restaurants more than one time which is valuable for the recommendation system. 

In [10]:
user_id = pd.DataFrame(df.user_id.value_counts().head(10))
user_id.reset_index(level=0, inplace = True)
user_id.columns = ['user_id', 'count']
user_id

,user_id,count
0,QxTNaHoxTL8f7jAK5nwQ1g,101
1,5aZX8bTiD0k9vR60SG588Q,76
2,GRY2acZtI5q4P1KdCWhcUQ,66
3,LcWOq7p7Mhtv9hIlDrhy9A,62
4,Y59HQNazSLR1EUMUmMOaFg,61
5,NWLqOKl0Vxi7qK-6EoBLbg,60
6,A5dqSwriUs8cV4DEzS_V9A,59
7,svFycHjXZYpNVutZ_0_gDQ,59
8,dl4ENy4Bk6-lCu59A8vxbg,57
9,94V-snVUg2Gd-OPd7rcZEg,54


**Table 4: Top 10 users left most reviews**

#### - user_name
Since different customers might have same name, in order to reduce the ambigurity, I will use user_id instead of user name in the recommendation system. There is nothing to do with the user_name column.

#### - user_rating
I will convert the string into numerical format as I mentioned and include user_rating variable to calculate the weights or scores the given customer rated. **Table 5** shows the count of number of each rating star in the Hoboken_restaurants_reviews dataset.

In [11]:
df.user_rating.unique()
df.user_rating = df.user_rating.replace({'5.0 star rating':5,
                                         '4.0 star rating':4, 
                                         '3.0 star rating':3,
                                         '2.0 star rating':2,
                                         '1.0 star rating':1
                                        })
rating = pd.DataFrame(df.user_rating.value_counts().head())
rating.reset_index(level=0, inplace = True)
rating.columns=['rating', 'count']
rating

,rating,count
0,5,27702
1,4,23140
2,3,10916
3,1,6608
4,2,6245


**Table 5: Count of rating**

#### - Restaurant_name
The restaurant's names are unique in this dataset. So I will use the restaurant name represent each restaurant in the following part. There are 302 different restaurants in the dataset in total. **Table 6 and Table 6.1** shows that some restaurants received more than 2 thousands revies in Yelp and some only received 1 or 2. This information might influence the accuracy of the recommendation system, since some types of recommendation methods will be limited by data sparsity. A restaurant with too seperated reviews might not be selected as a recommended result.

In [12]:
print('Number of restaurants:', len(df.restaurant_name.unique()))

Number of restaurants: 302


In [13]:
restaurant_name = df.groupby(df['restaurant_name']).user_id.count()
restaurant_name = pd.DataFrame(restaurant_name)
restaurant_name = restaurant_name.reset_index()
restaurant_name = restaurant_name.rename(index=str, columns={"user_id": "count"})
top_10 = restaurant_name.sort_values(by=['count'], ascending=False).head(10)
top_10

,restaurant_name,count
177,Morimoto,2740
261,The Spotted Pig,2640
153,Los Tacos No.1,2100
258,The Park,1640
287,Wafels & Dinges,1500
46,Catch,1480
90,Employees Only,1300
15,Artichoke Basille’s Pizza & Bar,1300
71,Del Posto,1260
93,Fig & Olive,1220


**Table 6: Restaurants received most reviews**

In [14]:
restaurant_name.sort_values(by=['count']).head(10)

,restaurant_name,count
165,Manhattan Bar Grill & Lounge,1
18,Azteca Taqueria Restaurant,1
159,MONDO - DOGS,1
81,El Cantante,1
289,Wah Yoan,1
92,Field House Grill,1
1,52 Restaurant,1
87,El Salvador Restaurant,1
240,Taquitos Mexicanos,2
99,Funjabi,2


**Table 6.1: Restaurants received least reviews**

#### -  restaurant_rating
I will convert the restaurant_rating column into numerical data. **Table 7** shows the distribution of different rating scores. The restaurant_rating variable might influence the customer's decision making, so I will take this variable into account in the Neural Network method.

In [15]:
print(df.restaurant_rating.unique())

df.restaurant_rating = df.restaurant_rating.replace({'5.0 star rating':5,
                                                     '4.5 star rating':4.5,
                                                     
                                                     '4.0 star rating':4,
                                                     '3.5 star rating':3.5,
                                                     '3.0 star rating':3,
                                                     '2.5 star rating':2.5,
                                                     '2.0 star rating':2,
                                                     '1.5 star rating':1.5,
                                                     '1.0 star rating':1
                                                    })
restaurant_rating = df.groupby(df['restaurant_rating']).user_id.count()
restaurant_rating = pd.DataFrame(restaurant_rating)
restaurant_rating = restaurant_rating.reset_index()
restaurant_rating = restaurant_rating.rename(index=str, columns={"user_id": "count"})
restaurant_rating = restaurant_rating.sort_values(by=['count'], ascending=False).head(15)
restaurant_rating

['4.0 star rating' '4.5 star rating' '3.5 star rating' '3.0 star rating'
 '2.5 star rating' '5.0 star rating' '1.5 star rating']


,restaurant_rating,count
4,4.0,31568
3,3.5,26130
5,4.5,8866
2,3.0,7263
6,5.0,484
1,2.5,267
0,1.5,33


**Table 7: Rating Distribution**

#### - Restaurant_price
I will convert the restaurant_price column into numerical data. **Table 8** shows the distribution of degree of restaurant's price. We could find that the level 2 degree is the most popular degree of restaurant's price.

In [16]:
print(df.restaurant_price.unique())

df.restaurant_price = df.restaurant_price.replace({'$':1,
                                                     '$$':2, 
                                                     '$$$':3,
                                                     '$$$$':4})

restaurant_price = df.groupby(df['restaurant_price']).user_id.count()
restaurant_price = pd.DataFrame(restaurant_price)
restaurant_price = restaurant_price.reset_index()
restaurant_price = restaurant_price.rename(index=str, columns={"user_id": "count"})
restaurant_price = restaurant_price.sort_values(by=['count'], ascending=False).head(15)
restaurant_price

['$$' '$' '$$$' nan '$$$$']


,restaurant_price,count
1,2.0,38323
2,3.0,19809
0,1.0,10567
3,4.0,5780


**Table 8: Price Distribution**

The restaurant_price variable might influence the customer's decision making. There are some missing restaurant_price for some restaurants, since yelp have not updated the price for these restaurants. I will use 2 to refill these cells according to the statistical result that there are most restaurants with restaurant price in level 2.

In [17]:
df.restaurant_price = df.restaurant_price.replace({None:2})
print(df.restaurant_price.unique())

[2. 1. 3. 4.]


#### - Restaurant_type
I will do data cleaning for the restaurant_type column. The restaurant_type variable is significant to influence customer select a restaurant. So I will focus on this variable in the Neural Network method. **Table 9** shows the most popular restaurant type in this dataset, 4000 users have left reviews to a 'sushi japanese bars' restaurant.

In [18]:
print('Sample:',df.restaurant_type.unique()[0:5])
print('Number of types:',len(df.restaurant_type.unique()))

Sample: ['Wine, Bars,, Italian,, Cocktail, Bars'
 'American, (New),, Gastropubs,, Irish, Pub' 'Sports, Bars,, Seafood'
 'Bakeries,, Coffee, &, Tea,, French' 'Bakeries,, Desserts,, Food, Stands']
Number of types: 230


In [19]:
li = []
for i in range(len(df)):
    item = df.restaurant_type.loc[i].replace(',', '')
    item = item.replace('&', '')
    item = item.replace('(','')
    item = item.replace(')','')
    item = item.lower()
    item = ' '.join(list(set(item.split())))
    li.append(item)
df['restaurant_type'] = li
restaurant_type = df.groupby(df['restaurant_type']).user_id.count()
restaurant_type = pd.DataFrame(restaurant_type)
restaurant_type = restaurant_type.reset_index()
restaurant_type = restaurant_type.rename(index=str, columns={"user_id": "count"})
restaurant_type = restaurant_type.sort_values(by=['count'], ascending=False).head(10)
restaurant_type

,restaurant_type,count
204,sushi bars japanese,4080
169,pizza italian,3348
133,italian,3070
151,mexican,2988
122,gastropubs burgers,2640
98,cocktail new spaces venues american bars event,1640
116,food belgian trucks waffles,1500
186,seafood bars fusion sushi asian,1480
10,american new bars,1451
103,cuban,1320


**Table 9: Restaurant Type with most reviews**

Now, I finish data cleaning and preprocessing part. I will store the cleaned dataset into 'Hoboken_restaurants_reviews_cleaned.csv'. This dataset will also be used in the method 4 - Neural Network methods.

In [20]:
df.head()

,user_id,user_name,user_rating,user_text,restaurant_name,restaurant_rating,restaurant_price,restaurant_type
0,dRuCO4NYO7zyAF8-CeJmZg,Jason L.,5,We booked Grand Vin as our brunch location to ...,Grand Vin,4.0,2.0,cocktail bars wine italian
1,f36YZ1cA291bNtMHXWtu1Q,Danyale W.,4,Sooooo for date night it was his turn to pick ...,Grand Vin,4.0,2.0,cocktail bars wine italian
2,-xYUKfWQTaB-7BeizsQA3w,Robin G.,5,Adorable little wine bar with outdoor seating ...,Grand Vin,4.0,2.0,cocktail bars wine italian
3,tt1vLgAP5UpRXAKJLT2KWg,Alec K.,4,One of the top restaurants in Hoboken. Well ma...,Grand Vin,4.0,2.0,cocktail bars wine italian
4,-K79Xep4lElqIChsJYWuiQ,Robbie O.,5,Great space- service is on point - short rib ...,Grand Vin,4.0,2.0,cocktail bars wine italian


<center>**Table 10: Clean dataset and Dataset for Method 4**</center>

In [21]:
df.to_csv('Hoboken_restaurants_reviews_cleaned.csv', index=False)

## Part 2. Data Preparation 
In this part I will prepare the data for co-occurence, collaborative filtering and matrix factorization methods. I will only keep the user_id, restaurant_name and user_rating column to build 3 basic recommendation system. **Table 11** shows a sample of this dataset.

In [22]:
df = pd.read_csv('Hoboken_restaurants_reviews_cleaned.csv')
train_raw = df[['user_id','restaurant_name', 'user_rating']]
train_raw.head()

,user_id,restaurant_name,user_rating
0,dRuCO4NYO7zyAF8-CeJmZg,Grand Vin,5
1,f36YZ1cA291bNtMHXWtu1Q,Grand Vin,4
2,-xYUKfWQTaB-7BeizsQA3w,Grand Vin,5
3,tt1vLgAP5UpRXAKJLT2KWg,Grand Vin,4
4,-K79Xep4lElqIChsJYWuiQ,Grand Vin,5


**Table 11: Sample of dataset**

Customers might rate more than one time for one restaurant. I will use the average rating scores for the cases that customer rated more than one time. This group-by table, **Table 12**, will also be used in collaborative filtering recommendation system.

In [23]:
gb_df = pd.DataFrame(train_raw.groupby(['user_id', 'restaurant_name']).mean())
gb_df = gb_df.reset_index()
gb_df.head()

,user_id,restaurant_name,user_rating
0,---xAZNw9fFPBoy7jmkA2A,Chef Of India,4
1,--68ZwhCrUJUmCXXkMTMKw,Prime Food Market,5
2,--8M2DZ9JkDwTveuRhLPTQ,Del Posto,5
3,--ARr3m5JsxaX3DTUVQW7w,Morimoto,2
4,--CZJeSIpxwQ0VULjnM57w,The Brass Rail,1


**Table 12: A group-by Dataset(Method 2)**

Based on the group-by dataset, I use pivot() function to convert it into a matrix like dataset. Each row represent each user and each column represent each restaurant. **Table 13** shows the sample of the matrix like dataset, which will be used for method 3, the matrix factorization recommendation system.

In [24]:
train_matrix_like_df = gb_df.pivot(index = 'user_id', columns ='restaurant_name', values = 'user_rating').fillna(0)
train_matrix_like_df.head()

restaurant_name,10th & Willow Bar & Grill,52 Restaurant,8th Street Tavern,Adoro Lei,Aether Game Cafe,Ahri’s Kitchen,Ainsworth Hoboken,Aldys Restaurant,Ali Baba,Amanda’s Restaurant,...,White Star Bar,Wicked Wolf Tavern,Willie McBrides,Yeung II Sushi Asian Cuisine,Zack’s Oak Bar & Restaurant,Zafra Kitchen,Zap’s,bwè kafe,honeygrow,奥田
user_id,,,,,,,,,,,,,,,,,,,,,
---xAZNw9fFPBoy7jmkA2A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--68ZwhCrUJUmCXXkMTMKw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--8M2DZ9JkDwTveuRhLPTQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ARr3m5JsxaX3DTUVQW7w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--CZJeSIpxwQ0VULjnM57w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<center>**Table 13: A Matrix Like dataset with rating(Method 3)**</center>

The **Table 14** is similar with the **Table 13**, the only difference is that I convert all of the rating scores into 1, which represent the customers attended to the restaurants. 

In [25]:
def get_dataset_co_occurence(df):
    #replace the rating with 1
    df = df.replace({5:1, 4:1, 3:1, 2:1})
    return df

In [26]:
Method_1 = get_dataset_co_occurence(train_matrix_like_df)
Method_1.head()

restaurant_name,10th & Willow Bar & Grill,52 Restaurant,8th Street Tavern,Adoro Lei,Aether Game Cafe,Ahri’s Kitchen,Ainsworth Hoboken,Aldys Restaurant,Ali Baba,Amanda’s Restaurant,...,White Star Bar,Wicked Wolf Tavern,Willie McBrides,Yeung II Sushi Asian Cuisine,Zack’s Oak Bar & Restaurant,Zafra Kitchen,Zap’s,bwè kafe,honeygrow,奥田
user_id,,,,,,,,,,,,,,,,,,,,,
---xAZNw9fFPBoy7jmkA2A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--68ZwhCrUJUmCXXkMTMKw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--8M2DZ9JkDwTveuRhLPTQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--ARr3m5JsxaX3DTUVQW7w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--CZJeSIpxwQ0VULjnM57w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<center>**Table 14: A matrix Like dataset without rating(Method 1)**</center>

After preparing the dataset for following part, I will store these dataset as 'Method_1_dataset.csv', 'Method_2_dataset.csv' and 'Method_3_dataset.csv'.

In [27]:
Method_1.to_csv('Method_1_dataset.csv',index=True)
gb_df.to_csv('Method_2_dataset.csv', index=False)
train_matrix_like_df.to_csv('Method_3_dataset.csv',index=True)

## Part 3  Recommendation System Methods:
#### In this part I will use following methods to build recommendation system model:
- co-occurrence matrices
- collaborative filtering
- matrix decomposition
- Neural Network

### Part3 - 1 co-occurrence Matrices

- Background:
This is the simpliest method I use to build the recommendation system. The underlying assumption is that the customers will be interested in the restaurants that the other customers who have similar interest.

- Algorithms/equations:  
    - Co-occurrence Matrix = A.T * A
    - Recommendation = Co-occurrence Matrix * u  
    Following chart will show a simple sample to apply co-occurrence matrix for recommendation system.
    ![](Image/co_occurrence_matrices.png)
    'A' represent the user-item matrix, which summarizes the interactions between users and items. The matrix is just from the raw data, with one row for each user, and one column for each item. 'A.T' is the transpose of this matrix. In 'A.T', rows means item and the column means user. In the first step, we should multiple 'A' and transposed 'A' and will get the co-occurrence Matrix, which is also the recommender to recommend the item to user.  Assume that we have a new user, this new user have bought the item 2. We multiple the co-occurrence matrix and the array represent the new user. Finally, we will obtain the recommendation result which shows we should recommend the Item 3 to this new user. The reason why we ignore the item 2 is that this user have already bought it.
    
- Tools:
    - Specifically, I will use numpy, pandas in python to build the recommendation model.

#### - Step 1: Data Loading
The data I input is the **Table 14**, a matrix like dataset without rating scores.

In [28]:
co_occurrence_dataset =  pd.read_csv('Method_1_dataset.csv')

#### - Step 2: build co-occurrence matrix
I use numpy to convert the matrix like dataset into numpy matrix and then obtain the transpose of this numpy matrix. Now I have two matrices, one original matrix and one transposed matrix. I multiple these two matrices and then eliminate the value in the diagonal cell of the result matrix to avoid repeadedly take attended restaurants into account. And then, I obtain the co-occurrence matrix like **chart 2**.

In [29]:
def get_co_occurrence_matrices(co_occurrence_dataset):
    #convert df to matix
    co_matrices =np.matrix(co_occurrence_dataset.iloc[:,1:])
    
    #A.T
    co_matrices_t = co_matrices.getT()
    
    #co_occurrence matrix
    co_occurrence_matrix = co_matrices_t * co_matrices
    
    #ignore the diagonal cell and avoid to repeadedly calculate the restaurants the user have already attended
    np.fill_diagonal(co_occurrence_matrix, 0)
    return co_occurrence_matrix

In [30]:
co_occurrence_matrix = get_co_occurrence_matrices(co_occurrence_dataset)
co_occurrence_matrix 

matrix([[ 0.,  0.,  9., ..., 11.,  3.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 9.,  0.,  0., ...,  6.,  0.,  0.],
        ...,
        [11.,  0.,  6., ...,  0., 13.,  0.],
        [ 3.,  0.,  0., ..., 13.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

**Chart 2: co-occurrence matrix**


#### - Step 3: Recommend restaurant for each user
In order to reach our target, to provide 3 more restaurants to each customer, I build a function could use co-occurrence matrix to provide recommended restaurants for each user. The underlying method is multiple the numpy array represented each user and the co-occurrence matrix we obtained in step 2.

In [31]:
def get_co_occurrence_result(co_occurrence_dataset, user_idx=None, user_id=None, top_n=None):
    
    """
    Args:
    
    co_occurrence_dataset: User and Items dataset
    co_occurrence_matrix: 
    
    """
    
    ##input is index
    if user_idx is not None:
        user = co_occurrence_dataset.iloc[user_idx, 1:]
    ##input is id
    elif user_id is not None:
        user = co_occurrence_dataset[co_occurrence_dataset.user_id == user_id].iloc[0, 1:]

    ##convert Series to array
    user_vector = np.array(user)

    ##get co occurence recommender
    co_occurrence_matrix = get_co_occurrence_matrices(co_occurrence_dataset)
    recommender = user_vector * co_occurrence_matrix
    # matrix to array
    recommender = np.array(recommender).reshape(-1, )
    # array to list
    recommender = recommender.tolist()

    ##export the result list for one user
    # create result dataframe
    user_result = pd.DataFrame(user)
    # append recommender list to this result dataframe
    user_result['recommender'] = recommender

    # check if the user have already attended this restaurant
    # only rank the restaurant they did not attend
    result_for_user = user_result[user_result.iloc[:, 0] != 1].sort_values(by='recommender', ascending=False)

    # create recommendation list with rating with n recommendations for each user
    result_for_user_restaurant = list(result_for_user.head(top_n).index.values)
    result_for_user_rating = list(result_for_user.recommender.head(top_n))
    recommend_restaurant_rating_li = []
    for i in range(len(result_for_user_restaurant)):
        recommend_rank_restaurant_rating = [str(i+1), result_for_user_restaurant[i], result_for_user_rating[i] ]
        recommend_restaurant_rating_li.append(recommend_rank_restaurant_rating)
    
    return recommend_restaurant_rating_li

I designed two options to select the user, user's index in the dataset or user's id. Following list show a sample result to recommend 3 more restaurants for a user with index 2. For this user, Morimoto is the first restaurant we will recommend with highest scores in 118.0 and so on.

In [32]:
given_user = get_co_occurrence_result(co_occurrence_dataset,  user_idx=2, user_id=None, top_n=3)
given_user

[['1', 'Morimoto', 118.0],
 ['2', 'The Spotted Pig', 106.0],
 ['3', 'EN Japanese Brasserie', 66.0]]

In addition, I build a function with using for loop to recommend the restaurant for each user in the dataset. So the co-occurrence matrix could provide recommend for each customer in the Hoboken_restaurants_reviews_cleaned.csv dataset.

In [33]:
def store_result(co_occurrence_dataset):
    # use for loop to get each user's recommendation list in the dataset
    co_occurrence_matrix = get_co_occurrence_matrices(co_occurrence_dataset)
    recommendation_li = []
    for i in range(len(co_occurrence_dataset)):
        # top_n = 3 according to the target of the project
        recommendation_for_user = get_co_occurrence_result(co_occurrence_dataset, user_id=co_occurrence_dataset.user_id[i], top_n=3)
        recommendation_li.append(recommendation_for_user)
        print(i)
        
    print('get list!')
    # create output dataset
    df = pd.read_csv('Method_1_dataset.csv')
    # first column
    df = pd.DataFrame(df.user_id)
    # second column
    df['recommendation'] = recommendation_li
    # save
    df.to_csv('co_occurence_result.csv', index = False)
    return None

In [34]:
#%%time
#store_result(co_occurrence_dataset)

#### - Step 4: Check the result
**Table 15** shows a sample of the recommendation result dataset for co-occurrence matrix method. Each user will receive recommendation of restaurants specifically.

In [35]:
co_occurence_result = pd.read_csv('Method_1_Co-Occurrence/co_occurrence_result.csv')
co_occurence_result.head()

,user_id,recommendation
0,---xAZNw9fFPBoy7jmkA2A,"[['1', 'Gino’s Pizzeria', 9.0], ['2', 'Noodlef..."
1,--68ZwhCrUJUmCXXkMTMKw,"[['1', 'The Cheesecake Factory', 3.0], ['2', '..."
2,--8M2DZ9JkDwTveuRhLPTQ,"[['1', 'Morimoto', 118.0], ['2', 'The Spotted ..."
3,--ARr3m5JsxaX3DTUVQW7w,"[['1', 'The Spotted Pig', 143.0], ['2', 'EN Ja..."
4,--CZJeSIpxwQ0VULjnM57w,"[['1', 'La Isla Restaurant', 48.0], ['2', 'Ama..."


<center>**Table 15: co-occurrence matrix result**</center>

These are two samples of recommendation result for the user with id '---xAZNw9fFPBoy7jmkA2A' and '--ARr3m5JsxaX3DTUVQW7w'. The model recommended three more restaurants for each user which stored in the recommendation column. Each cell is a list include three results. The first item (1, 2, 3) in the list means the ranking, the second item represent the name of restaurant, and the third item(9.0, 8.0, 6.0) means the weights/scores for each restaurant.

The weights/scores for different users are in different range according to the shape and values of co-occurrence matrix. For example, for the user with id '---xAZNw9fFPBoy7jmkA2A', the weights are 9.0, 8.0 and 6.0, while the weights are 143.0, 127.0 and 118.0 for the user with id '--ARr3m5JsxaX3DTUVQW7w'. 

In [36]:
print(co_occurence_result.loc[0,'recommendation'])

[['1', 'Gino’s Pizzeria', 9.0], ['2', 'Noodlefan', 8.0], ['3', 'Fox and Crow', 6.0]]


In [37]:
print(co_occurence_result.loc[3,'recommendation'])

[['1', 'The Spotted Pig', 143.0], ['2', 'EN Japanese Brasserie', 127.0], ['3', 'Del Posto', 118.0]]


### co-occurrence matrix summary:
In general, the co-occurrence matrix is easier to build than the other methods and it could build with dataset in any size, so it could be used for sparse dataset. However, the limitations for co-occurrence matrix is obvious. For example, It can only be built based on the user's past behaviors and it will ignore the user's rating for restaurants.

### Part 3 - 2 Collaborative Filtering
- My second recommendation system is collaborative filtering recommednation system. This method is tring to find similar users or items, evaluate the scores of those users/items, and recommend given user the items they will probably rate high. There are two different approaches in Collaborative Filtering: user-based collaborative filtering and item-based collaborative filtering. Basically, all of those two methods contains two steps:
    - First Step: Find out how many users/items in the database are similar to the given user/item.  
    - Second Step: Assess other users/items to predict what grade you would give the user of this product, given the total weight of the users/items that are more similar to this one.
I will include the user-based collaborative filtering in this project.     

- Algorithms/Equations:
    - Similarity Calculation: 
    The first step for collaborative filtering is calculate the similarity between the given user and the users in the dataset. There are multiple different methods to calculate the similarity and I will compare three popular methods, cosine similarity, jaccard similarity and pearson similarity.
    <script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=default"></script>
        1. cosine similarity:
        $${\text{Cosine similarity}}=\cos(\theta )={\mathbf {A} \cdot \mathbf {B}  \over \|\mathbf {A} \|\|\mathbf {B} \|}={\frac {\sum \limits _{i=1}^{n}{A_{i}B_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{A_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{B_{i}^{2}}}}}}$$

        2. Jaccard Similarity: 
        $${\text{Jaccard similarity}} = J(\mathbf{x}, \mathbf{y}) = \frac{\sum_i \min(x_i, y_i)}{\sum_i \max(x_i, y_i)}$$
   
        3. Pearson Similarity:
        $${\text{Pearson similarity}}={\frac {\sum _{i=1}^{n}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{{\sqrt {\sum _{i=1}^{n}(x_{i}-{\bar {x}})^{2}}}{\sqrt {\sum _{i=1}^{n}(y_{i}-{\bar {y}})^{2}}}}}$$
   
    - Recommend_items: 
    I will use weighted arithmetic mean according to the degree of similarity to fill empty cells in the recommendation-result table. The equation will be showed as follows:  
    
    $${\bar {x}}={\frac {w_{1}x_{1}+w_{2}x_{2}+\cdots +w_{n}x_{n}}{w_{1}+w_{2}+\cdots +w_{n}}}.$$
    
      
- Tools:  
I will use graphlab libaray in Python to build the recommendation system.

#### - Step 1: Data Loading
The data I input is the **Table 12**, a groub-by dataset for collaborative filtering.

In [2]:
df = pd.read_csv('Method_2_Collaborative_Filtering/Method_2_dataset.csv')

Since I use the graphlab library to build the collaborative filtering recommendation, I convert the pandas dataframe into graphlab dataframe, which is similar with the pandas dataframe but is suitable for function in graphlab library. In order to evaluate the performance of different methods to calculate similarity, I split the dataset into training set and test set with a ratio in 80/20.

In [3]:
train_data, test_data = train_test_split(df, test_size=0.2)
train_data_gl = graphlab.SFrame(train_data)
test_data_gl = graphlab.SFrame(test_data)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\manim\AppData\Local\Temp\graphlab_server_1525842408.log.0


This non-commercial license of GraphLab Create for academic use is assigned to nman@stevens.edu and will expire on April 25, 2019.


#### - Step 2: calculate similarity 
In this part I will test three different methods to calculate similarity and compare the precision and recall for each method.
The three methods will be showed as follows:
    1. cosine similarity; 
    2. Jaccard Similarity; 
    3. Pearson Similarity.
The definition of precision and recall will be showed as follows:
- Recall: It represent the ratio of items that a user likes were actually recommended.
- Precision: It means the user actually liked how many items out of all the recommended items.

#### - Cosine similarity
Similarity is the cosine of the angle between the 2 vectors of the item vectors of A and B. $${\text{Cosine similarity}}=\cos(\theta )={\mathbf {A} \cdot \mathbf {B}  \over \|\mathbf {A} \|\|\mathbf {B} \|}={\frac {\sum \limits _{i=1}^{n}{A_{i}B_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{A_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{B_{i}^{2}}}}}}$$
![](Image/cosine.png)
<center>**Table 16: Cosine precision and recall**</center>

In [4]:
Cosine_model = graphlab.item_similarity_recommender.create(train_data_gl, user_id='user_id', 
                                                           item_id='restaurant_name', 
                                                           target='user_rating', 
                                                           similarity_type='cosine',
                                                           verbose=False)

Recsys training: model = item_similarity

In **Table 16**, along with the increase of training epoches, the scores of precision and recall are enhance accordingly. In the cutoff 10, the precision is 0.0170 and recall is 0.1286.

#### - Jaccard Similarity
Jaccard similarity is based on the number of users which have rated item A and B divided by the number of users who have rated either A or B.clicked.$${\text{Jaccard similarity}} = J(\mathbf{x}, \mathbf{y}) = \frac{\sum_i \min(x_i, y_i)}{\sum_i \max(x_i, y_i)}$$
![](Image/jaccard.png)
<center>**Table 17: Jaccard precision and recall**</center>

In [5]:
Jaccard_model = graphlab.item_similarity_recommender.create(train_data_gl, user_id='user_id', 
                                                            item_id='restaurant_name', 
                                                            target='user_rating', 
                                                            similarity_type='jaccard',
                                                            verbose=False)

Recsys training: model = item_similarity

**Table 17** shows that along with the increase of training epoches, the scores of precision and recall are enhance accordingly. In the cutoff 10, the precision is 0.0167 and recall is 0.1274.

#### - Pearson Similarity
Pearson Similarity is the pearson coefficient between the two vectors. $${\text{Pearson similarity}}={\frac {\sum _{i=1}^{n}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{{\sqrt {\sum _{i=1}^{n}(x_{i}-{\bar {x}})^{2}}}{\sqrt {\sum _{i=1}^{n}(y_{i}-{\bar {y}})^{2}}}}}$$
![](Image/pearson.png)
<center>**Table 18: Pearson precision and recall**</center>

In [6]:
Pearson_model = graphlab.item_similarity_recommender.create(train_data_gl, user_id='user_id', 
                                                            item_id='restaurant_name', 
                                                            target='user_rating', 
                                                            similarity_type='pearson',
                                                            verbose=False)

Recsys training: model = item_similarity

In [7]:
model_performance = graphlab.compare(test_data_gl, [Cosine_model, Jaccard_model, Pearson_model])

PROGRESS: Evaluate model M0


recommendations finished on 1000/11633 queries. users per second: 104998

recommendations finished on 2000/11633 queries. users per second: 120904

recommendations finished on 3000/11633 queries. users per second: 127329

recommendations finished on 4000/11633 queries. users per second: 132983

...

recommendations finished on 8000/11633 queries. users per second: 88653.5

recommendations finished on 9000/11633 queries. users per second: 89762.1

recommendations finished on 10000/11633 queries. users per second: 89049.6

recommendations finished on 11000/11633 queries. users per second: 87070.8


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    | 2.86541161638e-05 | 6.61248834549e-06 |
|   4    | 2.14905871228e-05 | 6.61248834549e-06 |
|   5    | 3.43849393965e-05 | 1.73577819069e-05 |
|   6    | 2.86541161638e-05 | 1.73577819069e-05 |
|   7    | 3.68410064963e-05 | 0.000103320130398 |
|   8    | 5.37264678071e-05 | 0.000174955420808 |
|   9    | 6.68596043822e-05 | 0.000303898943545 |
|   10   | 7.73661136422e-05 | 0.000475823640528 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

Model compare metric: precision_recall


In **table 18**, along with the increase of training epoches, the scores of precision and recall are enhance accordingly. In the cutoff 10, the precision is 0.0001 and recall is 0.0011.

So, with comparison of these three methods to calculate similarity, the cosine similarity perform better than the others, while the Jaccard similarity only slightly lower than cosine similarity. So either cosine similarity or jaccard similarity would be good for the collaborative filtering.

Now I will use cosine similarity and jaccard similarity to recommend restaurants for each customer in the dataset.

#### - Step 4: Recommend top 3 rating restaurants to each customer
Cosine Similarity Result looks like **Table 19**. Jaccard Similarity result looks like **Table 20**. **Table 21** is the Pearson similarity result. Each customer receive three recommended restaurants ranking by the score.

In [8]:
train_data, test_data = train_test_split(df, test_size=0)
train_data_gl = graphlab.SFrame(train_data)
test_data_gl = graphlab.SFrame(test_data)

In [9]:
Cosine_recommendation = Cosine_model.recommend(k=3,verbose=False)
Cosine_recommendation.print_rows(num_rows=6)

+------------------------+-------------------------------+-----------------+------+
|        user_id         |        restaurant_name        |      score      | rank |
+------------------------+-------------------------------+-----------------+------+
| fsq7a4Mog2pJ00er6DuHvw |         Los Tacos No.1        |  0.160737276077 |  1   |
| fsq7a4Mog2pJ00er6DuHvw |             Decoy             | 0.0817471146584 |  2   |
| fsq7a4Mog2pJ00er6DuHvw | Artichoke Basille’s Pizza ... | 0.0772345662117 |  3   |
| J_nwnpiV4ZqohauRp7XgEQ |         Los Tacos No.1        |  0.267895460129 |  1   |
| J_nwnpiV4ZqohauRp7XgEQ |             Decoy             |  0.136245191097 |  2   |
| J_nwnpiV4ZqohauRp7XgEQ | Artichoke Basille’s Pizza ... |  0.12872427702  |  3   |
+------------------------+-------------------------------+-----------------+------+
[113163 rows x 4 columns]



**Table 19: Cosine Result**

In [10]:
Cosine_recommendation.save('Cosine_recommendation_result.csv')

In [11]:
Jaccard_recommendation = Jaccard_model.recommend(k=3,verbose=False)
Jaccard_recommendation.print_rows(num_rows=6)

+------------------------+-------------------------------+-----------------+------+
|        user_id         |        restaurant_name        |      score      | rank |
+------------------------+-------------------------------+-----------------+------+
| fsq7a4Mog2pJ00er6DuHvw |         Los Tacos No.1        | 0.0233147740364 |  1   |
| fsq7a4Mog2pJ00er6DuHvw |             Decoy             | 0.0152284502983 |  2   |
| fsq7a4Mog2pJ00er6DuHvw | Artichoke Basille’s Pizza ... | 0.0133729577065 |  3   |
| J_nwnpiV4ZqohauRp7XgEQ |         Los Tacos No.1        | 0.0233147740364 |  1   |
| J_nwnpiV4ZqohauRp7XgEQ |             Decoy             | 0.0152284502983 |  2   |
| J_nwnpiV4ZqohauRp7XgEQ | Artichoke Basille’s Pizza ... | 0.0133729577065 |  3   |
+------------------------+-------------------------------+-----------------+------+
[113163 rows x 4 columns]



**Table 20：Jaccard Result**

In [12]:
Jaccard_recommendation.save('Jaccard_recommendation_result.csv')

In [13]:
Pearson_recommendation = Pearson_model.recommend(k=3,verbose=False)
Pearson_recommendation.print_rows(num_rows=6)

+------------------------+------------------------------+-------+------+
|        user_id         |       restaurant_name        | score | rank |
+------------------------+------------------------------+-------+------+
| fsq7a4Mog2pJ00er6DuHvw |       Gold Roast Cafe        |  5.0  |  1   |
| fsq7a4Mog2pJ00er6DuHvw |        52 Restaurant         |  5.0  |  2   |
| fsq7a4Mog2pJ00er6DuHvw | Luna’s Pizzeria & Restaurant |  5.0  |  3   |
| J_nwnpiV4ZqohauRp7XgEQ |       Gold Roast Cafe        |  5.0  |  1   |
| J_nwnpiV4ZqohauRp7XgEQ |        52 Restaurant         |  5.0  |  2   |
| J_nwnpiV4ZqohauRp7XgEQ | Luna’s Pizzeria & Restaurant |  5.0  |  3   |
+------------------------+------------------------------+-------+------+
[113163 rows x 4 columns]



**Table 21: Pearson Result**

In [14]:
Pearson_recommendation.save('Pearson_recommendation_result.csv')

I am curious about why the Pearson similarity result seems similar for different users.

In [15]:
train_data.groupby(by='restaurant_name')['user_rating'].mean().sort_values(ascending=False).head(20)

restaurant_name
Manà Restaurant                    5.000000
Wah Yoan                           5.000000
Taquitos Mexicanos                 5.000000
Bocata’s Deli                      5.000000
Azteca Taqueria Restaurant         5.000000
Ujala Kabab                        5.000000
Manhattan Bar Grill & Lounge       5.000000
MONDO - DOGS                       5.000000
Ventura Restaurant                 5.000000
Terranova Deli                     5.000000
El Salvador Restaurant             5.000000
52 Restaurant                      5.000000
Gold Roast Cafe                    5.000000
Griot Cafe                         5.000000
Taste of Greece                    4.857143
Andrea Salumeria                   4.840000
Toscana                            4.789474
Royal Grill Halal Food             4.785714
Salerno Salumeria                  4.700000
Losurdo’s Italian Bakery & Deli    4.700000
Name: user_rating, dtype: float64

In [16]:
Cosine_recommendation_result = pd.read_csv('Cosine_recommendation_result.csv')
Jaccard_recommendation_result = pd.read_csv('Jaccard_recommendation_result.csv')
Pearson_recommendation_result = pd.read_csv('Pearson_recommendation_result.csv')

We could find that all the results are almost same when using pearson model. Since all the recommended restaurant have an average rating of 5. So the pearson recommender is not accurate enough. 


#### Collaborative Filtering Summary:

The collaborative filtering method seems better than the co-occurrence matrix, since it take the user's rating into account, which means it would be more relative to customers' real demand and it is still easy to understand. In addition, the collaborative filtering system will perform better and better with more and more users in the dataset. However,  it also only focus on the privious user's behaviors like the co-occurrence matrix method, so it doesn't have the ability to update accordingly. Moreover, it will be limited by data sparsity. If we could not collect enough data and the data sparsity is too low, the collaborative filtering will not perform well.




### Part 3 - 3 Matrix Factorization
- Background: The Matrix Factorization is the third method I used to build recommendation system. The assumption of matrix factorization is that each user have tendency to like different features of item. For example, in Hoboken restaurant dataset, the features might be theme and style of the restaurant, environment, service quality, food quality of restaurants and so on. Different user have different expectation and preferences for different features. In addition, each item, restaurant in this case, have features in different degree. We assume that the user will rate higher to a new restaurant with more features this user like and will rate lower to a new restaurant with less features this user like. 

- Algorithms/equation:
    - Basic Algorithm:
     $$\begin{equation}\hat{r}_{ui}=\sum_{k=1}^{K}{U_{uk}V_{ki}}\label{lfm}\end{equation}$$
            - R_ui: user u's rating for item i
            - U_uk:item i's similarity with a feature k
            - V_ki:user u's preference for a feature k 
    - Algorithm with bias:
    $$\begin{equation}\hat{r}_{ui}=\sum_{k=1}^{K}{U_{uk}V_{ki}}+\mu+b_u+b_i\label{bias_lfm}\end{equation}$$
    - This algorithm could also include more parameters like customer's feedback, taste changes according to time and so on.

- Tools: 
    - I will use sklearn and numpy in python to build the matrix factorization recommendation system.
    - The input data looks like the example as follows:  
     ![](Image/collaborative_filtering_1.png)
     
    - And then I will seperate this user and item matrix into two marix look like follows:
     ![](Image/collaborative_filtering_2.png)
    - K means features in this case so each row in the users and features matrix will represent the customer's preference for different features and each column in the features and items matrix will represent the degree of a restaurant have such features.

#### - Step 1: Data Preparation
In this step, I will convert the input data, **Table 13: A matrix like table with rating scores** into a numpy matrix. 

In [2]:
def get_matrix(file_name):
    df = pd.read_csv(file_name)
    #convert df to matrix
    matrix =np.matrix(df.iloc[:,1:])
    return matrix

In [3]:
matrices =get_matrix('Method_3_dataset.csv')
matrices

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

#### - Step 2: Extract features k  
I could use sklearn library to extract features k and seperate the matrix into two matrices, the user and features matrix as well as the features and items matrix. In this part, we reduce the dimension of original matrices and got two new matrices.
    - new matrix 1: represent user i's preference of different features k
    - new matrix 2: represent the item j's similarity to features k

In [4]:
def get_features_matrix(matrices, features_no):
    #NMF is a function could seperate one user and item matrix to two matrix, user and feature matrix as well as item and feature matrix.
    k_model = NMF(n_components=features_no, init='random', random_state=0)
    W = k_model.fit_transform(matrices)
    H = k_model.components_
    return W, H

After building the function, I randomly use features_no= 10 at the first try to create two new matrices. The shape of first matrix is (44949, 10) and second matrix is (10, 302).

In [38]:
W, H = get_features_matrix(matrices, 10)
print('Shape of matrix 1:', W.shape)
print('Shape of matrix 2:', H.shape)

#### - Step 3: Recommender
In this part, I will get the recommender matrix by dot product of these two matrices with using numpy.dot()function. The recommender matrix will look like **table 22**.

In [8]:
def get_recommender_df( W, H):
    df = pd.read_csv('Method_3_dataset.csv')
    #dot multiple user and feature matrix and item and feature matrix
    predict_matrix = np.matrix(np.dot(W,H))
    result_df = pd.DataFrame(predict_matrix)
    result_df.columns = df.columns[1:]
    result_df.insert(0, 'user_id', df.iloc[:, 0])
    return result_df

In [10]:
result_df = get_recommender_df( W, H)
result_df.head()

,user_id,10th & Willow Bar & Grill,52 Restaurant,8th Street Tavern,Adoro Lei,Aether Game Cafe,Ahri’s Kitchen,Ainsworth Hoboken,Aldys Restaurant,Ali Baba,...,White Star Bar,Wicked Wolf Tavern,Willie McBrides,Yeung II Sushi Asian Cuisine,Zack’s Oak Bar & Restaurant,Zafra Kitchen,Zap’s,bwè kafe,honeygrow,奥田
0,---xAZNw9fFPBoy7jmkA2A,0.003127,8.309046e-07,0.000713,0.000139,0.000200,0.000706,0.001725,0.000039,0.001400,...,0.002331,0.003011,0.001201,0.002167,0.004118,0.004056,0.000054,0.003183,0.001825,2.226274e-08
1,--68ZwhCrUJUmCXXkMTMKw,0.000378,9.981413e-08,0.000086,0.000032,0.000024,0.000096,0.000209,0.000005,0.000168,...,0.000285,0.000365,0.000145,0.000260,0.000495,0.000487,0.000007,0.000386,0.000220,0.000000e+00
2,--8M2DZ9JkDwTveuRhLPTQ,0.000000,0.000000e+00,0.000000,0.005092,0.000000,0.000156,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.013652,0.000000,0.000000,0.000000,0.000000e+00
3,--ARr3m5JsxaX3DTUVQW7w,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.001011,0.000159,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000532,0.000000,0.001081,0.000000e+00
4,--CZJeSIpxwQ0VULjnM57w,0.007670,2.037205e-06,0.001752,0.000336,0.000491,0.001732,0.004230,0.000096,0.003434,...,0.005721,0.007383,0.002947,0.005315,0.010097,0.009941,0.000132,0.007803,0.004473,7.965329e-07


<center>**Table 22: recommender matrix**</center>

#### - Step 4: Export the Result
In order to recommend new restaurants to each customer, I will remove the score for the restaurant each customer have already attended. And then each customer will receive three recommended restaurants. A sample result will look like following list:

In [11]:
def remove_attended_restaurant(result_df):
    df = pd.read_csv('Method_3_dataset.csv')
    for i in range(len(df)):
        attended_res_li = None
        # if attend, then the cell equal to 0
        attended_res_li = list(df.iloc[i, 1:][df.iloc[i, 1:] != 0].index)
        for res in attended_res_li:
            result_df.loc[i, res] = 0
    print('Remove Attended Restaurant!')
    return None

In [12]:
remove_attended_restaurant(result_df)

Remove Attended Restaurant!


In [13]:
def recommend_for_user(result_df, top_n, idx=None, user_id = None):
    if idx is not None:
        recommend_restaurant = list(result_df.iloc[idx,1:].sort_values(ascending=False).head(top_n).index)
        recommend_scores = list(result_df.iloc[idx,1:].sort_values(ascending=False).head(top_n))
    elif user_id is not None:
        recommend_restaurant = list(result_df[result_df.user_id == user_id].iloc[0, 1:].sort_values(ascending=False).head(top_n).index)
        recommend_scores = list(result_df[result_df.user_id == user_id].iloc[0,1:].sort_values(ascending=False).head(top_n))
    else:
        print('Unknown Error!')
    
    recommend_rank_restaurant_scores_li = []
    for i in range(len(recommend_restaurant)):
        item = [str(i+1), recommend_restaurant[i], recommend_scores[i]]
        recommend_rank_restaurant_scores_li.append(item)
        
    return recommend_rank_restaurant_scores_li

In [14]:
re_li = recommend_for_user(result_df, top_n=3, idx=1, user_id = None)

In [15]:
re_li

[['1', 'The Cuban Restaurant and Bar', 0.0017525621561418668],
 ['2', 'La Isla Restaurant', 0.0015292799793646238],
 ['3', 'Morimoto', 0.0014210628091436324]]

I build a function with using for loop to recommend the restaurants for each customer in the dataset and save the result as 'matrix_factorization_result.csv'.

In [16]:
def export_result(result_df):
    recommendation_li = []
    for i in range(len(result_df)):
        recommend_for_each_user = recommend_for_user(result_df, top_n=3, idx=i, user_id=None)
        recommendation_li.append(recommend_for_each_user)
        print(i)
    print('Done!')
    export_df = pd.DataFrame(result_df.user_id)
    export_df['Recommendation'] = recommendation_li
    export_df.to_csv('matrix_factorization_result.csv')
    return None

In [17]:
#%%time
#export_result(result_df)

#### - Step 5 Check the result
The **Table 23** shows a sample result of matrix factorization recommendation. The first item in the result list represent the ranking, the second item is the restaurant's name and the third item is the scores of each restaurant.

In [18]:
matrix_factorization_result = pd.read_csv('Method_3_Matrix_Factorization/matrix_factorization_result.csv')
matrix_factorization_result.head()

,Unnamed: 0,user_id,Recommendation
0,0,---xAZNw9fFPBoy7jmkA2A,"[['1', 'The Cuban Restaurant and Bar', 0.01458..."
1,1,--68ZwhCrUJUmCXXkMTMKw,"[['1', 'The Cuban Restaurant and Bar', 0.00175..."
2,2,--8M2DZ9JkDwTveuRhLPTQ,"[['1', 'Perry St', 0.23603528958327633], ['2',..."
3,3,--ARr3m5JsxaX3DTUVQW7w,"[['1', 'STK Downtown', 0.037760289982801606], ..."
4,4,--CZJeSIpxwQ0VULjnM57w,"[['1', 'The Cuban Restaurant and Bar', 0.03576..."


In [19]:
matrix_factorization_result.loc[0, 'Recommendation']

"[['1', 'The Cuban Restaurant and Bar', 0.01458922419511693], ['2', 'La Isla Restaurant', 0.012734421654190975], ['3', 'Pilsener Haus & Biergarten', 0.00954516675514815]]"

<center>**Table 23: A sample of matrix factorization result**</center>

#### Matrix Factorization Summary：

In general, matrix factorization method is more elegant than previous two methods. It could easily apply to masive dataset and we could add more new parameter(bias, historical feedback, time changes and so on) to the algorithms accordindly. However, the data sparisity will also limit its performance.


### Part 3 - 4. Deep Learning - neural network:
I have already build three recommendation systems with popular recommendation algorithms. Now, I will try to use deep learning method - Neural Networks to build a recommendation system. The objective is same as previous methods. I will recommend 3 resturants each customer most likely to be interested to them.

Neural Network method simulate the processes that human or animal recognized the object.  An artificial neural network includes one input layer, at least one hidden layer with nodes(kernel) and bias,  one output layer including defined target classes. Each layers and nodes will be randomly generated when launching the training process. Or if we apply transfer learning, the layers and nodes could be like a pre-trained model. Input variables will be normalized firstly, multiple with each neural weights, sum together, plus an bias and connect with the next layer. The target of each training process is trying to reduce the differences between real labels and the predicted results. We could use sigmoid, adam or other methods to optimize the reducing process. The revised information will transfer back to each layer and nodes as well as modify the weights and bias. Finally, we will get a neural network with relatively accurate weights and bias for each node on each layer.

![](Image/NN.png)

Before fiting the variable into neural network, we should normalize the input variables for each row first, since the Neural Network Algorithm assume that the input variable is an number between 0 to 1.

Tools: 
- I will use sklearn in python to build the NN recommendation system.

Model design:
- Label:
    - Restaurants Name (302 classes in total)

- According to the EDA part, I will include some variables in the model. 
- Input variables:
    - user_rating
    - restaurant rating
    - restaurant price
    - restaurant type
    
- The first three variables are numerical data and the restaurant type is data in text format. So I will also apply NLP to convert the restaurant_variables into the data type that NN could recognize.

- I design this model because the Neural Network model will return a restaurant that most likely match with the input variables. In a situation that accuracy eaqual to 100%, NN model will return the label (restaurant) we inputed. At the same time, it will also give a probability for each class in the dataset, which will show how does one restaurant is likely to be the accurate one. In another word, in order to build a recommendation system based on NN, I could rank both of these classes(302 restaurants) by the probability rate. And then, I could remove the accurate label in the ranking list and return other classes from higer probability to lower. So, the top n restaurants in this ranking will be the restaurants that are most similar to the accurate one.

### Part 3 - 4 - 1: Data Preparation
**Table 24** is the dataset I use to build the Neural Network recommendation system. I will use the restaurant_name as the label and use user_rating, restaurant_rating, restaurant_price and restaurant_type as the independent variables.

In [2]:
df = pd.read_csv('Method_4_Neural_Network/Hoboken_restaurants_reviews_cleaned.csv')

In [3]:
nn_df = df[['user_id','restaurant_name','user_rating','restaurant_rating','restaurant_price','restaurant_type']]
nn_df.head()

,user_id,restaurant_name,user_rating,restaurant_rating,restaurant_price,restaurant_type
0,dRuCO4NYO7zyAF8-CeJmZg,Grand Vin,5,4.0,2.0,cocktail bars wine italian
1,f36YZ1cA291bNtMHXWtu1Q,Grand Vin,4,4.0,2.0,cocktail bars wine italian
2,-xYUKfWQTaB-7BeizsQA3w,Grand Vin,5,4.0,2.0,cocktail bars wine italian
3,tt1vLgAP5UpRXAKJLT2KWg,Grand Vin,4,4.0,2.0,cocktail bars wine italian
4,-K79Xep4lElqIChsJYWuiQ,Grand Vin,5,4.0,2.0,cocktail bars wine italian


<center>**Table 24: Neural Network input data**</center>

#### Step 1: Normalized numerical data
In this step, I use 'max and min' normalized method to encode the user_rating, restaurant_rating and restaurant_price column into normalized value.

In [4]:
def normalize_variable(nn_df,variable_names):
    variables_li = []
    for variable in variable_names:
        variables_li.append(nn_df[variable])
    encoder = Normalizer()
    to_array = np.asarray(variables_li)
    norm_result = encoder.fit_transform(to_array)
    norm_result = norm_result.T
    return norm_result

In [5]:
variable_names = ['user_rating', 'restaurant_rating', 'restaurant_price']
norm_result = normalize_variable(nn_df,variable_names)

#### Step 2: Normalized text data by applying NLP- tf-idf
In this step, I use 'tf-idf' to convert the restaurant_type data into weights. 

In [6]:
stemmer = SnowballStemmer("english")
tokenizer = RegexpTokenizer("[a-z']+")

def tokenize(text):
    tokens = tokenizer.tokenize(text)
    return [stemmer.stem(t) for t in tokens] 

def get_tf(data, idf, max_df=1.0, min_df=1, ngram_range=(1,1)):
    if idf:
        """Convert a collection of raw documents to a matrix of TF-IDF features."""
        m = TfidfVectorizer(max_df=max_df, min_df=min_df, stop_words='english', ngram_range=ngram_range, tokenizer=tokenize,lowercase=True)
    d = m.fit_transform(data)
    return m, d

In [7]:
tfidf_m, tfidf_d = get_tf(nn_df['restaurant_type'], idf=True, max_df=0.5, min_df=10)

#### Step 3: Combine step 1 and step 2 and store the new matrix like dataframe
**Table 25** shows the new matrix like dataframe. Each row represent each customer. The 0 to 128 columns(129 in total) represent 129 features of restaurant type and the weight of preferences that each customer tend to like each feature. User_rating, restaurant_rating and restaurant_type columns are the encoded value according to original dataframe. Output column is the restaurant_name and will also be the label for each observation.

In [8]:
def get_nn_df(nn_df, norm_result, tfidf_d, variable_names):
    norm_result = pd.DataFrame(norm_result)
    tfidf_d = pd.DataFrame(tfidf_d.toarray())
    for index, name in enumerate(variable_names):
        tfidf_d[name] = norm_result.iloc[:,index]
    tfidf_d.index = nn_df.user_id
    tfidf_d['output'] = list(nn_df.restaurant_name)
    return tfidf_d

In [9]:
df = get_nn_df(nn_df, norm_result, tfidf_d, variable_names)
df.to_csv('Method_4_Neural_Network/normalized_neural_network_dataset.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,user_rating,restaurant_rating,restaurant_price,output
user_id,,,,,,,,,,,,,,,,,,,,,
dRuCO4NYO7zyAF8-CeJmZg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309521,0.0,0.0,...,0.0,0.0,0.0,0.670605,0.0,0.0,0.004578,0.003842,0.003032,Grand Vin
f36YZ1cA291bNtMHXWtu1Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309521,0.0,0.0,...,0.0,0.0,0.0,0.670605,0.0,0.0,0.003663,0.003842,0.003032,Grand Vin
-xYUKfWQTaB-7BeizsQA3w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309521,0.0,0.0,...,0.0,0.0,0.0,0.670605,0.0,0.0,0.004578,0.003842,0.003032,Grand Vin
tt1vLgAP5UpRXAKJLT2KWg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309521,0.0,0.0,...,0.0,0.0,0.0,0.670605,0.0,0.0,0.003663,0.003842,0.003032,Grand Vin
-K79Xep4lElqIChsJYWuiQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.309521,0.0,0.0,...,0.0,0.0,0.0,0.670605,0.0,0.0,0.004578,0.003842,0.003032,Grand Vin


<center>**Table 25: Neural Network dataset**</center>

### Part 3 - 4 - 2: Build Neural Network model and tune to find a best model

#### Step 1: Data loading and spliting
The first target in this step is to figure out a best model, so I split the input data into two set with ratio, 80/20.

In [10]:
X = df.iloc[:,:-1]
y = df.output
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=222)

#### Step 2: Build the model
At the first try, I randomly generate a neural network model with following parameters: 
    - hidden layer siz e= (16,6)  
    - solver = adam  
    - learning rate = 0.001

After training the model with training set and predicting the model with test set, the accuracy rate is 86.28% for the test set.

In [12]:
%%time
nn= MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(16, 6), random_state=1, learning_rate='adaptive', max_iter=1000)
nn = nn.fit(X_train, y_train)
#Predict
nn_prediction = nn.predict(X_test)
nn_acc = accuracy_score(y_test, nn_prediction)
print('Accuracy:',nn_acc)

Accuracy: 0.8628962004958789
Wall time: 3min 10s


#### Step 3:Tuning
In order to get a better model, I tune some parameters and obtain a better model with 87.77% accuracy rate and parameters as follows:
    - hidden layer siz e= (25,6)  
    - solver = adam  
    - learning rate = 0.005

In [30]:
%%time
#Tune
nn= MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(16, 6), random_state=1, learning_rate='adaptive', max_iter=10000)
param_grid = {'hidden_layer_sizes':[(16,3), (20,3),(20,6),(25,3),(25,6)], 
             'learning_rate_init': [0.0001, 0.0005, 0.001, 0.005]}
CV_nn = GridSearchCV(nn, param_grid,refit = True)
CV_nn.fit(X_train, y_train)
print (CV_nn.best_params_)

c:\programdata\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


{'hidden_layer_sizes': (25, 6), 'learning_rate_init': 0.005}
Wall time: 2h 56min 34s


In [31]:
nn_best = CV_nn.best_estimator_
nn_best = nn_best.fit(X_train, y_train)
nn_best_prediction = nn_best.predict(X_test)
nn_best_acc = accuracy_score(y_test, nn_best_prediction)
print('Accuracy:',nn_best_acc)

Accuracy: 0.8777055551832741


In [13]:
nn_best = nn

In [14]:
X_train = df.iloc[:,:-1]
y_train = df.output

#### Step 4: Get recommendation result with best estimator
Following table shows a sample result of the neural network model.

In [15]:
def get_recommendation_df(model, X_test):
    probability = model.predict_proba(X_test)
    recommendation_df = pd.DataFrame({'restaurant_name':model.classes_,'probability':probability[0]})
    return recommendation_df

In [16]:
def get_output(top_n, y_test,recommendation_df):
    top_n_df = recommendation_df[recommendation_df.restaurant_name != y_test].sort_values(by='probability', ascending=False).head(top_n)

    recommend_restaurant = list(top_n_df.loc[:,'restaurant_name'])
    recommend_score = list(top_n_df.loc[:,'probability'])
    recommendation_li = []
    for i in range(len(recommend_restaurant)):
        rank_restaurant_score = [str(i+1), recommend_restaurant[i], recommend_score[i]]
        recommendation_li.append(rank_restaurant_score)
    return recommendation_li

In [17]:
top_n = 3
X_test = np.asarray(X_train.iloc[8888,:])
X_test = X_test.reshape(-1,1).T
y_test = y_train.iloc[8888]
recommendation_df = get_recommendation_df(nn_best, X_test)
recommendation_df.head()

,probability,restaurant_name
0,1.504943e-37,10th & Willow Bar & Grill
1,4.619613e-58,52 Restaurant
2,5.929570e-134,8th Street Tavern
3,7.670625e-54,Adoro Lei
4,2.107623e-145,Aether Game Cafe


In [18]:
output = get_output(top_n, y_test,recommendation_df)
output

[['1', 'Augustino’s', 0.022578243355864183],
 ['2', 'Hudson Clearwater', 0.0003665265326626242],
 ['3', 'The Lobster Place', 0.00035943076450474433]]

In [19]:
def store_the_recommendation(X_train, y_train,nn_df, top_n, nn_best):
    export_df = pd.DataFrame(nn_df.user_id)
    recommendation_li = []
    for i in range(len(X_train)):
        X_test = np.asarray(X_train.iloc[i, :])
        X_test = X_test.reshape(-1, 1).T
        y_test = y_train.iloc[i]
        recommendation_df = get_recommendation_df(nn_best, X_test)
        output = get_output(top_n, y_test, recommendation_df)
        recommendation_li.append(output)
    export_df['Recommendation'] = recommendation_li
    export_df.to_csv('Neural_Network_result.csv', index=False)
    return None

In [20]:
#%%time
#store_the_recommendation(X_train, y_train ,nn_df, top_n, nn_best)

#### Step 5: Check the result
Following table shows the final result that recommend 3 restaurants to each customers. The shape of the result is similar with the co-occurrence result and matrix factarization result. The only different is that the third item means the probability that the recommended restaurant match with the restaurants the user attended.

In [21]:
NN_result = pd.read_csv('Method_4_Neural_Network/Neural_Network_result.csv')
NN_result.head()

,user_id,Recommendation
0,dRuCO4NYO7zyAF8-CeJmZg,"[['1', 'Barbuto', 0.0034586612971036304], ['2'..."
1,f36YZ1cA291bNtMHXWtu1Q,"[['1', 'Barbuto', 0.0032257489325294257], ['2'..."
2,-xYUKfWQTaB-7BeizsQA3w,"[['1', 'Barbuto', 0.0034586612971036304], ['2'..."
3,tt1vLgAP5UpRXAKJLT2KWg,"[['1', 'Barbuto', 0.0032257489325294257], ['2'..."
4,-K79Xep4lElqIChsJYWuiQ,"[['1', 'Barbuto', 0.0034586612971036304], ['2'..."


In [22]:
NN_result.loc[0, 'Recommendation']

"[['1', 'Barbuto', 0.0034586612971036304], ['2', 'Zack’s Oak Bar & Restaurant', 0.00043441651480210933], ['3', 'The Ear Inn', 0.0002991082358883752]]"

#### Neural Network Summary:
Neural Network recommendation system is different from previous three methods. The underlying concept is to predict the probability a restaurant matched with a customer's interests. This method could include more and take into account more parameters like customer's gender, their consumer behaviors, their social network relationship and so on. Also, along with more and more observations are using to feed the model, the accuracy of the model will be better and better. In addition, we could tune the model to find better parameters to build the neural network model. There are still space to tune and find a better model. However, the neural network recommendation system also have limitation that it could not perform well for a sparse dataset.

## Part 4: Comparison of the Recommendation System methods
In real world, there are multiple parameters to evaluate the performance of recommendation system, such as customer's satisfaction, diversity of recommendation, creativity, innovation, surprice degree, real time updated and cold start, profit of project. However, this project was built in offline sandbox, it is hard to use the online evaluation to detect the performance of these methods. 

So I will compare these four recommendation system methods through describing their limitaions, advantages and application scenarios.

- Co-occurrencce Matrix
    - Advantage： Easy to understand; Start with any size dataset
    - Limitation： Only focus on previous behaviors; ignore customer's rating
- Collaborative Filtering
    - Advantage：Perform well in masive dataset; Take customer's rating into account
    - Limitation: Only focus on previous behaviors; rely on customer's rating ; Limited by data sparsity
- Matrix Factorization
    - Advantage：Take more parameters like bias, customer's feed back, changes of taste into account
    - Limitation: Limited by data sparsity
- Neural Network |
    - Advantage：Easy to add more independent variables to build the NN model; Performance could be elevated with more data; Performance could be enhanced by tuning
    - Limitation: Limited by data sparsity

# V. Project Conclusion (Ni)
Our main target of this project is to explore the insight from customer reviews from Yelp.com. We have two data sources, 1. Customer Reviews of Restaurants at Hoboken (74, 611 rows). We build a Web Scraper to scrape this dataset. 2. Full dataset yelp provided (321 million rows). we use yelp open source API to collect this dataset.

In addition, we have tree sub objectives. The first one is predicting stars based on customer’s reviews and historical data. We use Random Forest algorithm in to predict the result. Our second sub objective is figuring out popular topics customer cared about, which will benefit owners of restaurants to improve their service quality. In this part, we applied NLP & Sentiment Analysis with using CV, tf-idf, LDA methods and machine Learning algotithms like Logistic Regression and Random Forest. The thrid sub target is recommending top 3 restaurants to each customer in the Hoboken dataset. In this part, we use four different recommendation methods including Co-occurrence Matrix, Collaborative Filtering, Matrix Factorization and a deep learning method Neural Network. 

In general, we reach the main target we setup and figure out the insight from reviews data. We provide scores prediction, help owners of restaurants modify their services and products and create profit for both restaurateurs and yelp.com.

# VI. Reference:
- Recommendation System part:
    1. Collaborative Filtering Recommender Based on Co-occurrence Matrix - Marjan Sterjev
    2. Moviex.ai - Movie recommendations with Deep Learning
    3. Deep Neural Networks for YouTube Recommendations - Paul Covington, Jay Adams, Emre Sargin
    4. Matrix Factorization Techniques for recommender system - Yehuda Koren, Robert Bell, Chris Volinsky
    5. https://learnforeverlearn.com/cooccurrence/
    6. https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/
    7. https://hub.packtpub.com/building-recommendation-engine-spark/
